In [40]:
using LinearAlgebra;
using SpecialFunctions;
using ControlSystems;
using Plots;
using VectorizedRoutines.Matlab

## LegT Measure


In [41]:
function legt_ss(dim::Integer, theta::Real)
    B = (2 * [0:dim-1;] .+ 1) .^ (1 / 2)
    A = -inv(theta) .* (B * B') .* Matrix{Integer}((-1) .^ triu([0:dim-1;] * ones(dim)' - ones(dim) * [0:dim-1;]'))

    legt = ss(A, B, Matrix(I, dim, dim), 0)
    return legt
end

legt_ss (generic function with 1 method)

In [ ]:
function build_LegT(N::Integer, lambda::Real=1)
    q = 1:N
    [k, n] = meshgrid(q, q)
    case = (n - k)^(-1)

    B = (2 * [0:dim-1;] .+ 1) .^ (1 / 2)
    A = -inv(theta) .* (B * B') .* Matrix{Integer}((-1) .^ triu([0:dim-1;] * ones(dim)' - ones(dim) * [0:dim-1;]'))

    legt = ss(A, B, Matrix(I, dim, dim), 0)
    return legt
end

## LagT Measure


In [42]:
function lagt_ss(dim::Integer, alpha::Real=0, beta::Real=1)
    A = -tril(ones(dim, dim), -1) - ((1 + beta) / 2) * I
    B = Vector{Float64}(1:dim)
    B = gamma.((B .+ alpha)) ./ (gamma.(B) .* gamma(alpha + 1))

    L = exp.(0.5 .* (loggamma.((B .+ beta)) .- loggamma.(B)))
    A = inv.(L) .* A .* L
    B = inv.(L) .* B .* exp.(-0.5 .* loggamma.((B .+ beta))) .* beta^((1 - alpha) / 2)

    lagt = ss(A, B, Matrix(I, dim, dim), 0)
    return lagt
end

function lagt_ss(dim::Integer, beta::Real=1)
    return lagt_ss(dim, 0, beta)
end

function lagt_ss(dim::Integer)
    return lagt_ss(dim, 1)
end

lagt_ss (generic function with 3 methods)

## LegS Measure

In [43]:
function legs_ss(dim::Integer)
    q = [0:dim-1;]
    r = 2 * q .+ 1
    M = -(tril(ones(dim) * r') - diagm(q))
    T = sqrt(diagm(r))
    A = T * M * inv(T)
    B = diag(T)

    legs = ss(A, B, Matrix(I, dim, dim), 0)
    return legs
end

legs_ss (generic function with 1 method)

## Visualizations

In [44]:
anim = @animate for i = 1:60
    legt = legt_ss(i,1);
    y, t, x = impulse(legt, 1.5)
    plot(t,x', xlabel="Time", xlims=(0, 1.5), ylimit=(-3, 3), title=("HiPPO-LegT Unit Impulse Response\nθ=1, n="*string(i)), lab=nothing)
end

gif(anim, "HiPPO-LegT_Unit_Impulse_Response.gif", fps = 15);
legt = legt_ss(15,1)

┌ Info: Saved animation to 
│   fn = /home/beegass/Documents/Coding/s4former/notebooks/notebooks-jl/HiPPO-LegT_Unit_Impulse_Response.gif
└ @ Plots /home/beegass/.julia/packages/Plots/E2187/src/animation.jl:126


StateSpace{Continuous, Float64}
A = 
 -1.0                  1.7320508075688772   -2.23606797749979     2.6457513110645907   -3.0                  3.3166247903554     -3.605551275463989    3.872983346207417    -4.123105625617661     4.358898943540674   -4.58257569495584     4.795831523312719   -5.0                  5.196152422706632   -5.385164807134504
 -1.7320508075688772  -2.9999999999999996    3.872983346207417   -4.58257569495584      5.196152422706632   -5.744562646538029    6.244997998398397   -6.7082039324993685    7.1414284285428495   -7.54983443527075     7.937253933193771   -8.306623862918073    8.660254037844386   -9.0                  9.327379053088814
 -2.23606797749979    -3.872983346207417    -5.000000000000001    5.916079783099617    -6.708203932499369    7.416198487095663   -8.06225774829855     8.660254037844387    -9.219544457292887     9.746794344808965  -10.246950765959598   10.723805294763608  -11.180339887498949   11.618950038622252  -12.041594578792296
 -2.64575

In [45]:
anim = @animate for i = 1:60
    legs = legs_ss(i);
    y, t, x = impulse(legs, 1.5)
    plot(t,x', xlabel="Time", xlims=(0, 1.5), ylimit=(-3, 3), title=("HiPPO-LegS Unit Impulse Response\nθ=1, n="*string(i)), lab=nothing)
end

gif(anim, "HiPPO-LegS_Unit_Impulse_Response.gif", fps = 15);
legs = legs_ss(15)

┌ Info: Saved animation to 
│   fn = /home/beegass/Documents/Coding/s4former/notebooks/notebooks-jl/HiPPO-LegS_Unit_Impulse_Response.gif
└ @ Plots /home/beegass/.julia/packages/Plots/E2187/src/animation.jl:126


StateSpace{Continuous, Float64}
A = 
 -1.0                  0.0                   0.0                   0.0                  0.0                  0.0                  0.0                  0.0                  0.0                  0.0                  0.0                  0.0                  0.0                 0.0                  0.0
 -1.7320508075688772  -2.0                   0.0                   0.0                  0.0                  0.0                  0.0                  0.0                  0.0                  0.0                  0.0                  0.0                  0.0                 0.0                  0.0
 -2.23606797749979    -3.8729833462074175   -3.0                   0.0                  0.0                  0.0                  0.0                  0.0                  0.0                  0.0                  0.0                  0.0                  0.0                 0.0                  0.0
 -2.6457513110645907  -4.582575694955841    -5.9160797830996

In [46]:
anim = @animate for i = 1:60
    lagt = lagt_ss(i, 0, 1);
    y, t, x = impulse(lagt, 1.5)
    plot(t,x', xlabel="Time", xlims=(0, 1.5), ylimit=(-3, 3), title=("HiPPO-LagT Unit Impulse Response\nθ=1, n="*string(i)), lab=nothing)
end

gif(anim, "HiPPO-LagT_Unit_Impulse_Response.gif", fps = 15);
lagt = lagt_ss(15, 0, 1)

┌ Info: Saved animation to 
│   fn = /home/beegass/Documents/Coding/s4former/notebooks/notebooks-jl/HiPPO-LagT_Unit_Impulse_Response.gif
└ @ Plots /home/beegass/.julia/packages/Plots/E2187/src/animation.jl:126


StateSpace{Continuous, Float64}
A = 
 -1.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -1.0  -1.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -1.0  -1.0  -1.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -1.0  -1.0  -1.0  -1.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1